<a href="https://colab.research.google.com/github/claredavies/ArtificialIntelligience/blob/main/ComputationalIntelligienceAssignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [19]:
import numpy

"""
The y=target is to maximize this equation ASAP:
    y = w1x1+w2x2+w3x3+w4x4+w5x5+6wx6
    where (x1,x2,x3,x4,x5,x6)=(4,-2,3.5,5,-11,-4.7)
    What are the best values for the 6 weights w1 to w6?
    We are going to use the genetic algorithm for the best possible values after a number of generations.
"""


'\nThe y=target is to maximize this equation ASAP:\n    y = w1x1+w2x2+w3x3+w4x4+w5x5+6wx6\n    where (x1,x2,x3,x4,x5,x6)=(4,-2,3.5,5,-11,-4.7)\n    What are the best values for the 6 weights w1 to w6?\n    We are going to use the genetic algorithm for the best possible values after a number of generations.\n'

In [6]:
# genetic algorithm search of the one max optimization problem
from numpy.random import randint
from numpy.random import rand

# Functions

In [24]:
def cal_pop_fitness(equation_inputs, pop):
    # Calculating the fitness value of each solution in the current population.
    # The fitness function caulcuates the sum of products between each input and its corresponding weight.
    fitness = numpy.sum(pop*equation_inputs, axis=1)
    return fitness

In [25]:
def select_mating_pool(pop, fitness, num_parents):
    # Selecting the best individuals in the current generation as parents for producing the offspring of the next generation.
    parents = numpy.empty((num_parents, pop.shape[1]))
    for parent_num in range(num_parents):
        max_fitness_idx = numpy.where(fitness == numpy.max(fitness))
        max_fitness_idx = max_fitness_idx[0][0]
        parents[parent_num, :] = pop[max_fitness_idx, :]
        fitness[max_fitness_idx] = -99999999999
    return parents

In [26]:
def crossover(parents, offspring_size):
    offspring = numpy.empty(offspring_size)
    # The point at which crossover takes place between two parents. Usually it is at the center.
    crossover_point = numpy.uint8(offspring_size[1]/2)
    for k in range(offspring_size[0]):
        # Index of the first parent to mate.
        parent1_idx = k%parents.shape[0]
        # Index of the second parent to mate.
        parent2_idx = (k+1)%parents.shape[0]
        # The new offspring will have its first half of its genes taken from the first parent.
        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        # The new offspring will have its second half of its genes taken from the second parent.
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
    return offspring

In [27]:
def mutation(offspring_crossover):
    # Mutation changes a single gene in each offspring randomly.
    for idx in range(offspring_crossover.shape[0]):
        # The random value to be added to the gene.
        random_value = numpy.random.uniform(-1.0, 1.0, 1)
        offspring_crossover[idx, 4] = offspring_crossover[idx, 4] + random_value
    return offspring_crossover

# Genetic Algorithm

In [29]:
# Inputs of the equation.
equation_inputs = [4,-2,3.5,5,-11,-4.7]

# Number of the weights we are looking to optimize.
num_weights = 6

"""
Genetic algorithm parameters:
    Mating pool size
    Population size
"""

sol_per_pop = 8
num_parents_mating = 4

# Defining the population size.
pop_size = (sol_per_pop,num_weights) # The population will have sol_per_pop chromosome where each chromosome has num_weights genes.
#Creating the initial population.
new_population = numpy.random.uniform(low=-4.0, high=4.0, size=pop_size)
print(new_population)

[[ 2.61832214 -3.99731799 -1.70515754  2.19798391 -1.91423096  3.74265175]
 [-1.2643703   0.15755551 -2.55835812 -0.5175099  -1.34312371 -0.5116738 ]
 [ 0.06046519  2.55647861  0.25822079  2.70941315  3.5455193   2.49692549]
 [-1.41843321 -2.73686897 -0.04039812  3.5013187   2.03276987  0.50524197]
 [-0.70191959  2.2621204  -0.45699598 -0.56797535 -2.67840078 -0.76271729]
 [ 0.85648357  3.6318075  -0.19261317  0.58872801 -2.61298913 -3.39611127]
 [-1.62969513 -2.97072974  2.30087151  2.18980049 -3.58140327 -3.74570684]
 [ 2.85307885  2.46805499  1.68646399 -3.45268435 -0.14241239  2.43006369]]


In [36]:
def evolutionary_algorithm(num_generations):
  for generation in range(num_generations):
      print("Generation : ", generation)
      # Measing the fitness of each chromosome in the population.
      fitness = cal_pop_fitness(equation_inputs, new_population)

      # Selecting the best parents in the population for mating.
      parents = select_mating_pool(new_population, fitness, 
                                        num_parents_mating)

      # Generating next generation using crossover.
      offspring_crossover = crossover(parents,
                                        offspring_size=(pop_size[0]-parents.shape[0], num_weights))

      # Adding some variations to the offsrping using mutation.
      offspring_mutation = mutation(offspring_crossover)

      # Creating the new population based on the parents and offspring.
      new_population[0:parents.shape[0], :] = parents
      new_population[parents.shape[0]:, :] = offspring_mutation

      # The best result in the current iteration.
      print("Best result : ", numpy.max(numpy.sum(new_population*equation_inputs, axis=1)))

  # Getting the best solution after iterating finishing all generations.
  #At first, the fitness is calculated for each solution in the final generation.
  fitness = cal_pop_fitness(equation_inputs, new_population)
  # Then return the index of that solution corresponding to the best fitness.
  best_match_idx = numpy.where(fitness == numpy.max(fitness))

  print("Finished Iterations")
  print("Best solution : ", new_population[best_match_idx, :])
  print("Best solution fitness : ", fitness[best_match_idx])

  return new_population[best_match_idx, :]

# Execution

In [38]:
num_generations = 5

evolutionary_algorithm(num_generations)

Generation :  0
Best result :  134.30444802581837
Generation :  1
Best result :  134.30444802581837
Generation :  2
Best result :  138.95503532987385
Generation :  3
Best result :  142.72556515422528
Generation :  4
Best result :  150.93878864210043
Finished Iterations
Best solution :  [[[ -1.62969513  -2.97072974   2.30087151   2.18980049 -10.44629408
    -3.74570684]]]
Best solution fitness :  [150.93878864]


array([[[ -1.62969513,  -2.97072974,   2.30087151,   2.18980049,
         -10.44629408,  -3.74570684]]])